## CENSUS DATABASE 2011-2018

In [29]:
import pandas as pd
from census import Census
import requests

# Census API Key
from config import api_key

# provide the api key and the year to establish a session
c = Census(api_key, year=2018)

# Set an option to allow up to 300 characters to print in each column
pd.set_option('max_colwidth', 300)

In [31]:
# query for all tables
tables = c.acs5.tables()

# The tables variable contains a list of dicts, so we can convert directly to a dataframe
table_df = pd.DataFrame(tables)
print(f"Number of available tables: {len(table_df)}")
table_df.head()

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [19]:
# Filter database : Income
table_df[table_df['description'].str.contains("FAMILY")]

,description,name,variables
0,POVERTY STATUS IN THE PAST 12 MONTHS OF FAMILIES BY FAMILY TYPE BY SOCIAL SECURITY INCOME BY SUPPLEMENTAL SECURITY INCOME (SSI) AND CASH PUBLIC ASSISTANCE INCOME,B17015,https://api.census.gov/data/2017/acs/acs5/groups/B17015.json
2,POVERTY STATUS IN THE PAST 12 MONTHS OF FAMILIES BY FAMILY TYPE BY WORK EXPERIENCE OF HOUSEHOLDER AND SPOUSE,B17016,https://api.census.gov/data/2017/acs/acs5/groups/B17016.json
10,AGGREGATE INCOME DEFICIT (DOLLARS) IN THE PAST 12 MONTHS FOR FAMILIES BY FAMILY TYPE,B17011,https://api.census.gov/data/2017/acs/acs5/groups/B17011.json
13,POVERTY STATUS IN THE PAST 12 MONTHS OF FAMILIES BY HOUSEHOLD TYPE BY NUMBER OF PERSONS IN FAMILY,B17013,https://api.census.gov/data/2017/acs/acs5/groups/B17013.json
15,POVERTY STATUS IN THE PAST 12 MONTHS OF FAMILIES BY HOUSEHOLD TYPE BY NUMBER OF WORKERS IN FAMILY,B17014,https://api.census.gov/data/2017/acs/acs5/groups/B17014.json
21,POVERTY STATUS IN THE PAST 12 MONTHS OF FAMILIES BY FAMILY TYPE BY PRESENCE OF RELATED CHILDREN UNDER 18 YEARS BY AGE OF RELATED CHILDREN,B17010,https://api.census.gov/data/2017/acs/acs5/groups/B17010.json
46,POVERTY STATUS IN THE PAST 12 MONTHS OF RELATED CHILDREN UNDER 18 YEARS BY FAMILY TYPE BY AGE OF RELATED CHILDREN UNDER 18 YEARS,B17006,https://api.census.gov/data/2017/acs/acs5/groups/B17006.json
180,AGGREGATE NONFAMILY HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2017 INFLATION-ADJUSTED DOLLARS) BY SEX OF HOUSEHOLDER BY LIVING ALONE BY AGE OF HOUSEHOLDER,B19216,https://api.census.gov/data/2017/acs/acs5/groups/B19216.json
181,MEDIAN NONFAMILY HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2017 INFLATION-ADJUSTED DOLLARS) BY SEX OF HOUSEHOLDER BY LIVING ALONE BY AGE OF HOUSEHOLDER,B19215,https://api.census.gov/data/2017/acs/acs5/groups/B19215.json
182,AGGREGATE NONFAMILY HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2017 INFLATION-ADJUSTED DOLLARS),B19214,https://api.census.gov/data/2017/acs/acs5/groups/B19214.json


In [20]:
# Determine which table you're interested in
table_id = 'B19119'

# Capture the variables URL from the table_df
url = table_df.loc[table_df['name']==table_id, 'variables'].values[0]

# Make the API call
response = requests.get(url).json()

# convert the response to a DataFrame
variables = pd.DataFrame(response['variables']).transpose()

print(f"Number of available variables: {len(variables)}")
variables.head(15)

Number of available variables: 28


,concept,group,label,limit,predicateOnly,predicateType
B19119_005E,MEDIAN FAMILY INCOME IN THE PAST 12 MONTHS (IN 2017 INFLATION-ADJUSTED DOLLARS) BY FAMILY SIZE,B19119,Estimate!!Total!!5-person families,0,True,int
B19119_005M,MEDIAN FAMILY INCOME IN THE PAST 12 MONTHS (IN 2017 INFLATION-ADJUSTED DOLLARS) BY FAMILY SIZE,B19119,Margin of Error!!Total!!5-person families,0,True,int
B19119_006E,MEDIAN FAMILY INCOME IN THE PAST 12 MONTHS (IN 2017 INFLATION-ADJUSTED DOLLARS) BY FAMILY SIZE,B19119,Estimate!!Total!!6-person families,0,True,int
B19119_006M,MEDIAN FAMILY INCOME IN THE PAST 12 MONTHS (IN 2017 INFLATION-ADJUSTED DOLLARS) BY FAMILY SIZE,B19119,Margin of Error!!Total!!6-person families,0,True,int
B19119_007E,MEDIAN FAMILY INCOME IN THE PAST 12 MONTHS (IN 2017 INFLATION-ADJUSTED DOLLARS) BY FAMILY SIZE,B19119,Estimate!!Total!!7-or-more-person families,0,True,int
B19119_007M,MEDIAN FAMILY INCOME IN THE PAST 12 MONTHS (IN 2017 INFLATION-ADJUSTED DOLLARS) BY FAMILY SIZE,B19119,Margin of Error!!Total!!7-or-more-person families,0,True,int
B19119_001E,MEDIAN FAMILY INCOME IN THE PAST 12 MONTHS (IN 2017 INFLATION-ADJUSTED DOLLARS) BY FAMILY SIZE,B19119,Estimate!!Total,0,True,int
B19119_001M,MEDIAN FAMILY INCOME IN THE PAST 12 MONTHS (IN 2017 INFLATION-ADJUSTED DOLLARS) BY FAMILY SIZE,B19119,Margin of Error!!Total,0,True,int
B19119_002E,MEDIAN FAMILY INCOME IN THE PAST 12 MONTHS (IN 2017 INFLATION-ADJUSTED DOLLARS) BY FAMILY SIZE,B19119,Estimate!!Total!!2-person families,0,True,int
B19119_002M,MEDIAN FAMILY INCOME IN THE PAST 12 MONTHS (IN 2017 INFLATION-ADJUSTED DOLLARS) BY FAMILY SIZE,B19119,Margin of Error!!Total!!2-person families,0,True,int


In [21]:
variables[variables['predicateType']=='int'].head()

,concept,group,label,limit,predicateOnly,predicateType
B19119_005E,MEDIAN FAMILY INCOME IN THE PAST 12 MONTHS (IN 2017 INFLATION-ADJUSTED DOLLARS) BY FAMILY SIZE,B19119,Estimate!!Total!!5-person families,0,True,int
B19119_005M,MEDIAN FAMILY INCOME IN THE PAST 12 MONTHS (IN 2017 INFLATION-ADJUSTED DOLLARS) BY FAMILY SIZE,B19119,Margin of Error!!Total!!5-person families,0,True,int
B19119_006E,MEDIAN FAMILY INCOME IN THE PAST 12 MONTHS (IN 2017 INFLATION-ADJUSTED DOLLARS) BY FAMILY SIZE,B19119,Estimate!!Total!!6-person families,0,True,int
B19119_006M,MEDIAN FAMILY INCOME IN THE PAST 12 MONTHS (IN 2017 INFLATION-ADJUSTED DOLLARS) BY FAMILY SIZE,B19119,Margin of Error!!Total!!6-person families,0,True,int
B19119_007E,MEDIAN FAMILY INCOME IN THE PAST 12 MONTHS (IN 2017 INFLATION-ADJUSTED DOLLARS) BY FAMILY SIZE,B19119,Estimate!!Total!!7-or-more-person families,0,True,int


In [22]:
census_data = c.acs5.get(("NAME", "B19013_001E", "B01003_001E", "B01002_001E", "B19301_001E"), 
                         {'for': 'zip code tabulation area:*'})

census_data[:5]

[{'NAME': 'ZCTA5 00601',
  'B19013_001E': 11757.0,
  'B01003_001E': 17599.0,
  'B01002_001E': 38.9,
  'B19301_001E': 7041.0,
  'zip code tabulation area': '00601'},
 {'NAME': 'ZCTA5 00602',
  'B19013_001E': 16190.0,
  'B01003_001E': 39209.0,
  'B01002_001E': 40.9,
  'B19301_001E': 8978.0,
  'zip code tabulation area': '00602'},
 {'NAME': 'ZCTA5 00603',
  'B19013_001E': 16645.0,
  'B01003_001E': 50135.0,
  'B01002_001E': 40.4,
  'B19301_001E': 10897.0,
  'zip code tabulation area': '00603'},
 {'NAME': 'ZCTA5 00606',
  'B19013_001E': 13387.0,
  'B01003_001E': 6304.0,
  'B01002_001E': 42.8,
  'B19301_001E': 5960.0,
  'zip code tabulation area': '00606'},
 {'NAME': 'ZCTA5 00610',
  'B19013_001E': 18741.0,
  'B01003_001E': 27590.0,
  'B01002_001E': 41.4,
  'B19301_001E': 9266.0,
  'zip code tabulation area': '00610'}]

In [23]:
# Convert to DataFrame
census_pd = pd.DataFrame(census_data)

# Renaming columns to be more user-friendly
census_pd = census_pd.rename(columns={"B01003_001E": "Population",
                                      "B01002_001E": "Median Age",
                                      "B19013_001E": "Household Income",
                                      "B19301_001E": "Per Capita Income",
                                      "NAME": "Name", 
                                      "zip code tabulation area": "Zipcode"})

# Since Census doesn't provide the poverty rate, we can divide Poverty Count by Population to calculate it ourselves
#census_pd["Poverty Rate"] = 100 * census_pd["Poverty Count"].astype(int) / census_pd["Population"].astype(int)

# Reorder columns and only include ones we're interested in for the final DataFrame
census_pd = census_pd[["Zipcode", "Population", "Median Age", "Household Income",
                       "Per Capita Income"]]

# Visualize
print("Total number of zip codes in response: " + str(len(census_pd)))
census_pd.head(50)

Total number of zip codes in response: 33120


,Zipcode,Population,Median Age,Household Income,Per Capita Income
0,00601,17599.0,38.9,11757.0,7041.0
1,00602,39209.0,40.9,16190.0,8978.0
2,00603,50135.0,40.4,16645.0,10897.0
3,00606,6304.0,42.8,13387.0,5960.0
4,00610,27590.0,41.4,18741.0,9266.0
5,00612,62566.0,41.4,17744.0,10283.0
6,00616,10687.0,43.7,14918.0,9002.0
7,00617,24508.0,38.7,17157.0,8762.0
8,00622,7405.0,48.4,16727.0,11935.0
9,00623,42321.0,41.3,16401.0,9101.0


In [24]:
census_pd.to_csv("census_data_2018.csv", encoding="utf-8", index=False)